# Siglent SDG2000X Driver Test Notebook

This notebook tests the functionality of the `SDG2000X` driver class. It covers basic waveform generation, specific pulse configurations, arbitrary waveform uploading, and triggering.

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import matplotlib.pyplot as plt

# Import the driver as requested
from piec.drivers.awg.sdg2000 import SDG2000X
from piec.drivers.utilities import PiecManager

In [ ]:
pm = PiecManager()
pm.list_resources()

## 1. Connection and Identification
Set the `ADDRESS` variable below to match your specific instrument (USB or TCPIP).

In [ ]:
# REPLACE with your actual VISA resource address
# Example USB: 'USB0::0xF4EC::0x1101::#15::INSTR'
# Example LAN: 'TCPIP::192.168.1.10::INSTR'
ADDRESS = 'USB0::0xF4EC::0x1101::#15::INSTR'

try:
    awg = SDG2000X(ADDRESS)
    print("Connected successfully.")
    
    # Query Instrument ID
    idn = awg.idn()
    print(f"Instrument ID: {idn}")
    
    # Reset to known state
    awg.reset()
    print("Instrument Reset.")
    
except Exception as e:
    print(f"Connection Failed: {e}")

## 2. Basic Waveform Configuration
Testing Sine, Square, and Ramp waves on Channel 1.

In [ ]:
CHANNEL = 1

# 1. Setup a Sine Wave
print("Setting up Sine Wave...")
awg.set_waveform(CHANNEL, 'SIN')
awg.set_frequency(CHANNEL, 1000)      # 1 kHz
awg.set_amplitude(CHANNEL, 2.0)       # 2 Vpp
awg.set_offset(CHANNEL, 0.0)          # 0 V offset
awg.set_load_impedance(CHANNEL, 50)   # 50 Ohm load

# Enable Output
awg.output(CHANNEL, on=True)
print(f"Channel {CHANNEL} Output: ON (Sine, 1kHz, 2Vpp)")
time.sleep(2)

# 2. Setup Square Wave with Duty Cycle
print("Setting up Square Wave (20% Duty)...")
awg.set_waveform(CHANNEL, 'SQU')
awg.set_square_duty_cycle(CHANNEL, 20.0)
time.sleep(2)

# 3. Setup Ramp Wave with Symmetry
print("Setting up Ramp Wave (90% Symmetry)...")
awg.set_waveform(CHANNEL, 'RAMP')
awg.set_ramp_symmetry(CHANNEL, 90.0)
time.sleep(2)

# Turn off for safety before next test
awg.output(CHANNEL, on=False)
print(f"Channel {CHANNEL} Output: OFF")

## 3. Pulse Configuration
Testing pulse width, rise time, and fall time parameters.

In [ ]:
print("Configuring Pulse...")
awg.set_waveform(CHANNEL, 'PULS')
awg.set_frequency(CHANNEL, 500)       # 500 Hz

# Pulse Specifics
awg.set_pulse_width(CHANNEL, 200e-6)  # 200 microseconds
awg.set_pulse_rise_time(CHANNEL, 20e-9) # 20 ns
awg.set_pulse_fall_time(CHANNEL, 20e-9) # 20 ns
awg.set_pulse_delay(CHANNEL, 10e-6)     # 10 us delay

awg.output(CHANNEL, on=True)
print("Pulse Output Enabled.")
time.sleep(2)
awg.output(CHANNEL, on=False)

## 4. Arbitrary Waveform Generation
Generates a mathematical waveform in Python and uploads it to the instrument.

In [ ]:
# Generate data: A Damped Sine Wave
num_points = 16384
t = np.linspace(0, 10 * np.pi, num_points)
data_float = np.sin(t) * np.exp(-t/10)

# Normalize to signed 16-bit integer range (-32767 to 32767)
# The driver expects integers to pack into binary
max_val = 32000 # Keep slightly within range to be safe
data_int = (data_float * max_val).astype(int)

# Visualize what we are sending
plt.figure(figsize=(10, 4))
plt.plot(data_int[:1000])
plt.title("First 1000 points of Arb Waveform")
plt.show()

wave_name = "DampedSin"

print(f"Uploading Arbitrary Waveform '{wave_name}'...")
try:
    # 1. Upload the wave
    awg.create_arb_waveform(CHANNEL, wave_name, data_int)
    
    # 2. Select the wave (driver method create_arb_waveform also selects it, but testing explicit set)
    awg.set_arb_waveform(CHANNEL, wave_name)
    
    # 3. Set parameters for playback
    awg.set_frequency(CHANNEL, 1000)
    awg.set_amplitude(CHANNEL, 2.5)
    
    awg.output(CHANNEL, on=True)
    print(f"Arb Waveform '{wave_name}' output enabled.")
except Exception as e:
    print(f"Arb upload failed: {e}")

## 5. Triggering
Testing manual trigger output.

In [ ]:
# Setup for Burst to see trigger effect clearly
awg.set_waveform(CHANNEL, 'SIN')
awg.set_frequency(CHANNEL, 1000)

# Set Trigger Source to Manual
awg.set_trigger_source(CHANNEL, 'MAN')

# Send Trigger
print("Sending Manual Trigger...")
awg.output_trigger()
print("Trigger Sent.")

## 6. Cleanup

In [ ]:
awg.output(1, on=False)
awg.output(2, on=False)
print("Outputs disabled. Test complete.")